# Training and packaging Spacy

- In this notebook, we assume that we already have a usable vector space, contained in the `embeddings.txt` file. To see how we compile such a vector space, see the notebook `02-fastext-vectors.ipynb`

## Imports and utilities

In [ ]:
import os
import spacy
import json

In [ ]:
def make_dirs(path_or_uri: str, exist_ok: bool = True):
    return os.makedirs(path_or_uri, exist_ok=exist_ok)

## Configuration

In [ ]:
lang = 'nb'
n_dims = 300               # vector dimensions
n_vectors = 5000           # in production we use 50k
vector_arch = 'fasttext'   # just used in model name
version = '2.0.0'
batch_size = 2000
patience = 1000            # 10000 better for real modelling
                           # very important parameter: how long SpaCy will run the training process before early stopping
                           # we do not know how long it will really take...

pkg_name_short = f'nhst_{vector_arch}_{n_dims}_{n_vectors}'
pkg_name_full = f'{lang}_{pkg_name_short}-{version}'

vectors_dir = f'nhst_{n_dims}'
job_dir = f'job_data'

# ready to use GPU: install pytorch in environment
use_gpu = False
if use_gpu:
    gpu_id = 0
    gpu_allocator = '"pytorch"'  # double quotes needed
    spacy.require_gpu()
else:
    gpu_id = -1
    gpu_allocator = 'none'

## Init vectors

In [ ]:
# we already have a simple vector file... this should be computed in advance!
embeddings_local = 'embeddings.txt'

make_dirs(vectors_dir)

# run spacy init vectors
args = ['python', '-m', 'spacy', 'init', 'vectors', lang, embeddings_local, vectors_dir, '--prune', str(n_vectors),
        '--name', f'nhst_{n_dims}']

print(' '.join(args))  # copy output and run

In [ ]:
!python -m spacy init vectors nb embeddings.txt nhst_300 --prune 10000 --name nhst_300

## Write job-setup files 

In [ ]:
yml_config_str = '''
title: "Norwegian POS Tagging, Dependency Parsing (Universal Dependencies) and NER with Norne"
description: "Template to train a POS tagger, morphologizer, dependency parser amd named entity recogniser from a
[Universal Dependencies](https://universaldependencies.org/) corpus, in its Norne version.
It takes care of downloading the treebank, converting it to spaCy's format and training and evaluating the model."

vars:
  config: "default"
  lang: "{package_lang}"
  treebank: "norne"
  train_name: "no_bokmaal-ud-train"
  dev_name: "no_bokmaal-ud-dev"
  test_name: "no_bokmaal-ud-test"
  package_name: "{package_name}"
  package_version: "{package_version}"
  gpu: {gpu}

# These are the directories that the project needs. The project CLI will make sure that they always exist.
directories: ["assets", "corpus", "training", "metrics", "configs", "packages"]

assets:
  - dest: "assets/${{vars.treebank}}"
    git:
      repo: "https://github.com/ltgoslo/${{vars.treebank}}"  # "https://github.com/UniversalDependencies/${{vars.treebank}}"
      branch: "master"
      path: ""

workflows:
  all:
    - preprocess
    - train
    - evaluate
    - package

commands:
  - name: preprocess
    help: "Convert the data to spaCy's format"
    script:
      - "mkdir -p corpus/${{vars.treebank}}"
      - "python -m spacy convert assets/${{vars.treebank}}/ud/nob/${{vars.train_name}}.conllu corpus/${{vars.treebank}}/ --converter conllu --n-sents 10 --merge-subtokens"
      - "python -m spacy convert assets/${{vars.treebank}}/ud/nob/${{vars.dev_name}}.conllu corpus/${{vars.treebank}}/ --converter conllu --n-sents 10 --merge-subtokens"
      - "python -m spacy convert assets/${{vars.treebank}}/ud/nob/${{vars.test_name}}.conllu corpus/${{vars.treebank}}/ --converter conllu --n-sents 10 --merge-subtokens"
      - "mv corpus/${{vars.treebank}}/${{vars.train_name}}.spacy corpus/${{vars.treebank}}/train.spacy"
      - "mv corpus/${{vars.treebank}}/${{vars.dev_name}}.spacy corpus/${{vars.treebank}}/dev.spacy"
      - "mv corpus/${{vars.treebank}}/${{vars.test_name}}.spacy corpus/${{vars.treebank}}/test.spacy"
    deps:
      - "assets/${{vars.treebank}}/ud/nob/${{vars.train_name}}.conllu"
      - "assets/${{vars.treebank}}/ud/nob/${{vars.dev_name}}.conllu"
      - "assets/${{vars.treebank}}/ud/nob/${{vars.test_name}}.conllu"
    outputs:
      - "corpus/${{vars.treebank}}/train.spacy"
      - "corpus/${{vars.treebank}}/dev.spacy"
      - "corpus/${{vars.treebank}}/test.spacy"

  - name: train
    help: "Train ${{vars.treebank}}"
    script:
      - "python -m spacy train configs/${{vars.config}}.cfg --output training/${{vars.treebank}} --gpu-id ${{vars.gpu}} --paths.train corpus/${{vars.treebank}}/train.spacy --paths.dev corpus/${{vars.treebank}}/dev.spacy --nlp.lang=${{vars.lang}}"
    deps:
      - "corpus/${{vars.treebank}}/train.spacy"
      - "corpus/${{vars.treebank}}/dev.spacy"
      - "configs/${{vars.config}}.cfg"
    outputs:
      - "training/${{vars.treebank}}/model-best"

  - name: evaluate
    help: "Evaluate on the test data and save the metrics"
    script:
      - "python -m spacy evaluate ./training/${{vars.treebank}}/model-best ./corpus/${{vars.treebank}}/test.spacy --output ./metrics/metrics.json --gpu-id ${{vars.gpu}}"
    deps:
      - "training/${{vars.treebank}}/model-best"
      - "corpus/${{vars.treebank}}/test.spacy"
    outputs:
      - "metrics/metrics.json"

  - name: package
    help: "Package the trained model so it can be installed"
    script:
      - "python -m spacy package training/${{vars.treebank}}/model-best packages --name ${{vars.package_name}} --version ${{vars.package_version}} --force"
    deps:
      - "training/${{vars.treebank}}/model-best"
    outputs_no_cache:
      - "packages/${{vars.lang}}_${{vars.package_name}}-${{vars.package_version}}/dist/${{vars.lang}}_${{vars.package_name}}-${{vars.package_version}}.tar.gz"

  - name: clean
    help: "Remove intermediate files"
    script:
      - "rm -rf training/*"
      - "rm -rf metrics/*"
      - "rm -rf corpus/*"

'''

In [ ]:
# Based on Norwegian official package (large), plus GPU
exp_config_str = '''
[paths]
train = "corpus/norne/train.spacy"
dev = "corpus/norne/dev.spacy"
vectors = "../{vectors_dir}"
init_tok2vec = null

[system]
gpu_allocator = {allocator}
seed = 0

[nlp]
lang = "nb"
pipeline = ["tok2vec","morphologizer","parser","lemmatizer","senter","attribute_ruler","ner"]
disabled = ["senter"]
before_creation = null
after_creation = null
after_pipeline_creation = null
batch_size = {batch_size}
tokenizer = {{"@tokenizers":"spacy.Tokenizer.v1"}}
vectors = {{"@vectors":"spacy.Vectors.v1"}}

[components]

[components.attribute_ruler]
factory = "attribute_ruler"
scorer = {{"@scorers":"spacy.attribute_ruler_scorer.v1"}}
validate = false

[components.lemmatizer]
factory = "trainable_lemmatizer"
backoff = "orth"
min_tree_freq = 3
overwrite = false
scorer = {{"@scorers":"spacy.lemmatizer_scorer.v1"}}
top_k = 1

[components.lemmatizer.model]
@architectures = "spacy.Tagger.v2"
nO = null
normalize = false

[components.lemmatizer.model.tok2vec]
@architectures = "spacy.Tok2VecListener.v1"
width = ${{components.tok2vec.model.encode:width}}
upstream = "tok2vec"

[components.morphologizer]
factory = "morphologizer"
extend = false
label_smoothing = 0.0
overwrite = true
scorer = {{"@scorers":"spacy.morphologizer_scorer.v1"}}

[components.morphologizer.model]
@architectures = "spacy.Tagger.v2"
nO = null
normalize = false

[components.morphologizer.model.tok2vec]
@architectures = "spacy.Tok2VecListener.v1"
width = ${{components.tok2vec.model.encode:width}}
upstream = "tok2vec"

[components.ner]
factory = "ner"
incorrect_spans_key = null
moves = null
scorer = {{"@scorers":"spacy.ner_scorer.v1"}}
update_with_oracle_cut_size = 100

[components.ner.model]
@architectures = "spacy.TransitionBasedParser.v2"
state_type = "ner"
extra_state_tokens = false
hidden_width = 64
maxout_pieces = 2
use_upper = true
nO = null

[components.ner.model.tok2vec]
@architectures = "spacy.Tok2Vec.v2"

[components.ner.model.tok2vec.embed]
@architectures = "spacy.MultiHashEmbed.v2"
width = 96
attrs = ["NORM","PREFIX","SUFFIX","SHAPE"]
rows = [5000,1000,2500,2500]
include_static_vectors = true

[components.ner.model.tok2vec.encode]
@architectures = "spacy.MaxoutWindowEncoder.v2"
width = 96
depth = 4
window_size = 1
maxout_pieces = 3

[components.parser]
factory = "parser"
learn_tokens = false
min_action_freq = 30
moves = null
scorer = {{"@scorers":"spacy.parser_scorer.v1"}}
update_with_oracle_cut_size = 100

[components.parser.model]
@architectures = "spacy.TransitionBasedParser.v2"
state_type = "parser"
extra_state_tokens = false
hidden_width = 64
maxout_pieces = 2
use_upper = true
nO = null

[components.parser.model.tok2vec]
@architectures = "spacy.Tok2VecListener.v1"
width = ${{components.tok2vec.model.encode:width}}
upstream = "tok2vec"

[components.senter]
factory = "senter"
overwrite = false
scorer = {{"@scorers":"spacy.senter_scorer.v1"}}

[components.senter.model]
@architectures = "spacy.Tagger.v2"
nO = null
normalize = false

[components.senter.model.tok2vec]
@architectures = "spacy.Tok2Vec.v2"

[components.senter.model.tok2vec.embed]
@architectures = "spacy.MultiHashEmbed.v2"
width = 16
attrs = ["NORM","PREFIX","SUFFIX","SHAPE","SPACY"]
rows = [1000,500,500,500,50]
include_static_vectors = true

[components.senter.model.tok2vec.encode]
@architectures = "spacy.MaxoutWindowEncoder.v2"
width = 16
depth = 2
window_size = 1
maxout_pieces = 2

[components.tok2vec]
factory = "tok2vec"

[components.tok2vec.model]
@architectures = "spacy.Tok2Vec.v2"

[components.tok2vec.model.embed]
@architectures = "spacy.MultiHashEmbed.v2"
width = ${{components.tok2vec.model.encode:width}}
attrs = ["NORM","PREFIX","SUFFIX","SHAPE","SPACY","IS_SPACE"]
rows = [5000,1000,2500,2500,50,50]
include_static_vectors = true

[components.tok2vec.model.encode]
@architectures = "spacy.MaxoutWindowEncoder.v2"
width = 96
depth = 4
window_size = 1
maxout_pieces = 3

[corpora]

[corpora.dev]
@readers = "spacy.Corpus.v1"
path = ${{paths.dev}}
gold_preproc = false
max_length = 0
limit = 0
augmenter = null

[corpora.train]
@readers = "spacy.Corpus.v1"
path = ${{paths.train}}
gold_preproc = false
max_length = 0
limit = 0
augmenter = null

[training]
train_corpus = "corpora.train"
dev_corpus = "corpora.dev"
seed = ${{system:seed}}
gpu_allocator = ${{system:gpu_allocator}}
dropout = 0.1
accumulate_gradient = 1
patience = {patience}
max_epochs = 0
max_steps = 100000
eval_frequency = 1000
frozen_components = []
before_to_disk = null
annotating_components = []
before_update = null

[training.batcher]
@batchers = "spacy.batch_by_words.v1"
discard_oversize = false
tolerance = 0.2
get_length = null

[training.batcher.size]
@schedules = "compounding.v1"
start = 100
stop = 1000
compound = 1.001
t = 0.0

[training.logger]
@loggers = "spacy.ConsoleLogger.v1"
progress_bar = false

[training.optimizer]
@optimizers = "Adam.v1"
beta1 = 0.9
beta2 = 0.999
L2_is_weight_decay = true
L2 = 0.01
grad_clip = 1.0
use_averages = true
eps = 0.00000001
learn_rate = 0.001

[training.score_weights]
pos_acc = 0.14
morph_acc = 0.14
morph_per_feat = null
dep_uas = 0.0
dep_las = 0.29
dep_las_per_type = null
sents_p = null
sents_r = null
sents_f = 0.04
lemma_acc = 0.1
ents_f = 0.29
ents_p = 0.0
ents_r = 0.0
ents_per_type = null
speed = 0.0

[pretraining]

[initialize]
vocab_data = null
vectors = ${{paths.vectors}}
init_tok2vec = ${{paths.init_tok2vec}}
before_init = null
after_init = null

[initialize.components]

[initialize.components.lemmatizer]

[initialize.components.lemmatizer.labels]
@readers = "spacy.read_labels.v1"
path = "corpus/labels/trainable_lemmatizer.json"
require = false

[initialize.components.morphologizer]

[initialize.components.morphologizer.labels]
@readers = "spacy.read_labels.v1"
path = "corpus/labels/morphologizer.json"
require = false

[initialize.components.ner]

[initialize.components.ner.labels]
@readers = "spacy.read_labels.v1"
path = "corpus/labels/ner.json"
require = false

[initialize.components.parser]

[initialize.components.parser.labels]
@readers = "spacy.read_labels.v1"
path = "corpus/labels/parser.json"
require = false

[initialize.lookups]
@misc = "spacy.LookupsDataLoader.v1"
lang = ${{nlp.lang}}
tables = []

[initialize.tokenizer]
'''

In [ ]:
# save configuration strings to files
make_dirs(job_dir)
with open(f'{job_dir}/project.yml', 'w') as f:
    f.write(yml_config_str.format(package_lang=lang,
                                  package_name=pkg_name_short,
                                  package_version=version,
                                  gpu=gpu_id))

make_dirs(f'{job_dir}/configs')
with open(f'{job_dir}/configs/default.cfg', 'w') as f:
    f.write(exp_config_str.format(vectors_dir=vectors_dir,
                                  allocator=gpu_allocator,
                                  batch_size=batch_size,
                                  patience=patience))

## Downloading data assets for training

In [ ]:
args = ['python', '-m', 'spacy', 'project', 'assets', job_dir]

print(' '.join(args))  # copy output and run

In [ ]:
!python -m spacy project assets job_data

## Running job tasks

In [ ]:
args = ['python', '-m', 'spacy', 'project', 'run', 'all', job_dir]

print(' '.join(args))  # copy output and run

In [ ]:
!python -m spacy project run all job_data

## Saving output

- the necessary files are already saved, but we should move them to the final location

In [ ]:
# built automatically by Spacy, but we need it to copy the package
pkg_file = f'{job_dir}/packages/{pkg_name_full}/dist/{pkg_name_full}.tar.gz'
print(pkg_file)

# evaluation metrics
metrics_file = f'{job_dir}/metrics/metrics.json'
print(metrics_file)

In [ ]:
# check metrics
with open(f'{job_dir}/metrics/metrics.json') as f:
    metrics = json.load(f)
    for k, v in metrics.items():
        print('##', k, '##')
        print(v)
        print()